# CNN 예제

TensorFlow 2.0을 통해 이미지 데이터를 전처리하고, CNN을 사용하는 예제를 살펴봅니다.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

## Cats vs Dogs 데이터셋 로드

Cats vs Dogs 데이터셋을 로드합니다.

Cats vs Dogs 데이터셋은 TFDS에서 제공하는 이미지 분류 데이터셋으로,

3 color(RGB) 이미지로 강아지와 고양이 사진으로 구성되어 있습니다.

In [ ]:
dataset_name = 'cats_vs_dogs'
dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

## 로드한 데이터 확인

예시로 몇 개의 데이터만 확인해보겠습니다.

강아지, 고양이의 사진과 라벨 정보를 확인할 수 있습니다.

라벨 정보는 고양이 -> 0 / 강아지 -> 1 인 것을 확인 할 수 있습니다.

Cats vs Dogs Dataset의 세부적인 정보는 아래 링크에서 확인할 수 있습니다.

https://www.tensorflow.org/datasets/catalog/cats_vs_dogs

In [ ]:
import matplotlib.pyplot as plt

fig = tfds.show_examples(dataset, info)

## 데이터 전처리

이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

우선 데이터의 크기가 제각각 다르므로, 동일한 크기로 Resize를 실시해야하며,

0-255 사이의 pixel 값을 0-1 사이의 float 값으로 Normalize를 실시해야합니다.

<br>

여기서는 임의로 (224, 224) 크기로 Resize를 지정해주었습니다.

preprocess 함수를 전처리 함수로 선언한 뒤, dataset.map 을 이용하여 실제로 전처리를 진행하도록 합니다.


In [ ]:
def preprocess(features):
  
    img = features['image']
    label = features['label']
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32)
    img = img / 255.0

    return img, label

train_dataset = dataset.map(preprocess).batch(32)

## 네트워크 정의

tf.keras.models.Sequential을 이용해 CNN을 정의합니다.

CNN은 구조적으로 Feature Extractor와 Classifier로 구성됩니다.

Feature Extractor는 Conv2D와 MaxPooling2D 레이어로 구성되며,
Classifier는 Flatten과 Dense 레이어로 구성됩니다.

<br>

먼저, Feature Extractor를 만들기 위해, Conv2D 레이어가 필요합니다.

Conv2D 레이어는 filter 갯수, kernel 사이즈, activation 함수를 argument로 받습니다.

특히나 첫 레이어의 경우에는 입력되는 shape를 함께 지정해줍니다.

여기서 shape는 위에서 전처리를 해주었기 때문에 (224, 224, 3) 으로 지정합니다.

Conv2D 레이어 이후에는 MaxPooling2D 레이어를 추가합니다.

Conv2D와 MaxPooling2D를 번갈아가며 추가하는 것으로 Feature Extractor를 구성하도록 합니다.

<br>

2회 혹은 3회 가량 Conv2D와 MaxPooling2D를 연결한 다음, Classifier를 구성하기 위해 Flatten 레이어를 추가합니다.

Flatten 레이어를 거치고 나면, 모든 parameter들이 1차원 데이터로 구성됩니다.

1차원 데이터를 Dense 레이어를 통해 Fully Connected 레이어 계산을 해주도록 합니다.

<br>

그리고 마지막에 강아지 고양이의 이진 분류를 위해서 2개의 출력 값을 갖는 Dense 레이어를 배치하도록 합니다.

마지막 Dense 레이어의 activation 함수로는 softmax를 지정합니다.

<br>

Loss 함수를 SparseCategoricalCrossentropy로 지정하여 compile 합니다.

Loss 정보가 one-hot encoding([0, 1] 혹은 [1, 0] 형태)라면 CategoricalCrossentropy를 사용하면 됩니다.

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

model.compile(optimizer='RMSprop', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

## 학습

학습 후 결과를 확인합니다.

accuracy가 90% 이상 나오도록 위의 네트워크 구조 및 학습 epoch 수를 변경하시면 좋습니다.

In [ ]:
model.fit(train_dataset, epochs=5)